# A Good Modeling Example


## Contents

1. [Environment Initialisation](#Environment-Initialisation)
2. [Data Preparation](#Data-Preparation)
    1. [Synthetic Minority Oversampling Technique (SMOTE)](#Synthetic-Minority-Oversampling-Technique-%28SMOTE%29)
    2. [Correlation Reduction](#Correlation-Reduction)
3. [Model Construction](#Model-Construction)
    1. [Hyperparameter Optimisation](#Hyperparameter-Construction)
    2. [Final Model](#Final-Model)
    3. [Testing](#Testing)


## Environment Initialisation


In [ ]:
%matplotlib inline


In [ ]:
import copy
import math
import numbers
import os
import pickle
import sys


In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import numpy as np
import pandas as pd
import scipy as sp
import scipy.optimize as opt
import scipy.special as spec
import scipy.stats as stats


In [ ]:
import sklearn as skl
import sklearn.decomposition as skldec
import sklearn.linear_model as skllm
import sklearn.metrics as sklmet


In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa

import tensorflow.keras as keras
import tensorflow.keras.backend as K
import tensorflow.keras.callbacks as callbacks
import tensorflow.keras.layers as layers
import tensorflow.keras.losses as losses
import tensorflow.keras.metrics as metrics
import tensorflow.keras.models as models
import tensorflow.keras.optimizers as optimizers
import tensorflow.keras.utils as utils
import tensorflow_addons.losses as tfa_losses
import tensorflow_addons.metrics as tfa_metrics


In [ ]:
import hyperopt as hopt


In [ ]:
import networkx as nx


In [ ]:
import seaborn as sns


In [ ]:
plt.style.use('seaborn')
sns.set_theme()


The following versions were used in the development of this *Jupyter Notebook*:


In [ ]:
print(
    "Python version: " \
        f"{sys.version_info.major}." \
        f"{sys.version_info.minor}." \
        f"{sys.version_info.micro}"
)

print(f"Scipy version: {sp.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")
print(f"Matplotlib version: {mpl.__version__}")

print(f"scikit-learn version: {skl.__version__}")

print(f"Tensorflow version: {tf.__version__}")
print(f"Keras version: {keras.__version__}")

print(f"Hyperopt version: {hopt.__version__}")

print(f"NetworkX version: {nx.__version__}")

print(f"Seaborn version: {sns.__version__}")


In [ ]:
import utils as myutils


## Data Preparation


In [ ]:
data_root = 'data'
arrays_root = 'arrays'
models_root = 'models'


In [ ]:
df = pd.read_csv(
    os.path.join(data_root, 'scores.csv'),
    sep = ';',
    header = 0,
    index_col = None,
    decimal = ','
)


In [ ]:
labels = pd.read_csv(
    os.path.join(data_root, 'labels.csv'),
    sep = ';',
    header = 0,
    index_col = None,
    decimal = ','
)


In [ ]:
df = df.merge(labels, how = 'left', on = 'country')


In [ ]:
del labels


In [ ]:
df = df.reindex(
    [
        'year',
        'country',
        'label',
        'score',
        'nscore',
        'rank'
    ],
    axis = 1
).reset_index(drop = True).copy(deep = True)


In [ ]:
R = np.random.default_rng(2022)


In [ ]:
year = 1990


In [ ]:
X_train = list()
year_train = list()
time_train = list()
pos_train = list()
diverse_train = list()
y_train = list()

X_valid = list()
year_valid = list()
time_valid = list()
pos_valid = list()
diverse_valid = list()
y_valid = list()

for y in range(year - 10, year):
    trains = np.load(os.path.join(arrays_root, str(y), 'train.npy'))

    I = df.index[df.year == y]

    for i in I:
        c = str(np.asarray(df.loc[i, 'country']).item())
        a = np.asarray(df.loc[i, 'year']).item()
        s = np.asarray(df.loc[i, 'nscore']).item()

        x = np.load(os.path.join(arrays_root, str(y), c, 'taggram.npy'))
        t = np.load(os.path.join(arrays_root, str(y), c, 't.npy'))
        p = np.load(os.path.join(arrays_root, str(y), c, 'p.npy'))
        d = np.load(os.path.join(arrays_root, str(y), c, 'diverse.npy'))

        if np.asarray(df.loc[i, 'label']).item() in trains:
            X_train.append(x)
            year_train.append(np.full(x.shape[0], a, dtype = x.dtype))
            time_train.append(np.full(x.shape[0], t, dtype = x.dtype))
            pos_train.append(np.full(x.shape[0], p, dtype = x.dtype))
            diverse_train.append(np.full(x.shape[0], d, dtype = x.dtype))
            y_train.append(np.full(x.shape[0], s, dtype = x.dtype))
        else:
            X_valid.append(x)
            year_valid.append(np.full(x.shape[0], a, dtype = x.dtype))
            time_valid.append(np.full(x.shape[0], t, dtype = x.dtype))
            pos_valid.append(np.full(x.shape[0], p, dtype = x.dtype))
            diverse_valid.append(np.full(x.shape[0], d, dtype = x.dtype))
            y_valid.append(np.full(x.shape[0], s, dtype = x.dtype))

X_train = np.concatenate(X_train, axis = 0)
year_train = np.squeeze(np.concatenate(year_train, axis = 0))
time_train = np.squeeze(np.concatenate(time_train, axis = 0))
pos_train = np.squeeze(np.concatenate(pos_train, axis = 0))
diverse_train = np.squeeze(np.concatenate(diverse_train, axis = 0))
y_train = np.squeeze(np.concatenate(y_train, axis = 0))

X_valid = np.concatenate(X_valid, axis = 0)
year_valid = np.squeeze(np.concatenate(year_valid, axis = 0))
time_valid = np.squeeze(np.concatenate(time_valid, axis = 0))
pos_valid = np.squeeze(np.concatenate(pos_valid, axis = 0))
diverse_valid = np.squeeze(np.concatenate(diverse_valid, axis = 0))
y_valid = np.squeeze(np.concatenate(y_valid, axis = 0))


In [ ]:
I = R.permutation(y_train.shape[0])

X_train = X_train[I].copy()
year_train = year_train[I].copy()
time_train = time_train[I].copy()
pos_train = pos_train[I].copy()
diverse_train = diverse_train[I].copy()
y_train = y_train[I].copy()

I = R.permutation(y_valid.shape[0])

X_valid = X_valid[I].copy()
year_valid = year_valid[I].copy()
time_valid = time_valid[I].copy()
pos_valid = pos_valid[I].copy()
diverse_valid = diverse_valid[I].copy()
y_valid = y_valid[I].copy()


In [ ]:
feat_df_train = pd.DataFrame(
    np.concatenate(
        (
            X_train,
            np.expand_dims(year_train.astype(np.float32), axis = 1),
            np.expand_dims(time_train, axis = 1),
            np.expand_dims(pos_train, axis = 1),
            np.expand_dims(diverse_train, axis = 1),
            np.expand_dims(y_train, axis = 1)
        ),
        axis = 1
    ),
    columns = pd.Index(
        list(f"p_{i + 1:d}" for i in range(int(X_train.shape[1]))) +
            [ 'y', 't', 'p', 'd', 's' ],
        dtype = np.str_,
        name = 'feature'
    )
)

feat_df_valid = pd.DataFrame(
    np.concatenate(
        (
            X_valid,
            np.expand_dims(year_valid, axis = 1),
            np.expand_dims(time_valid, axis = 1),
            np.expand_dims(pos_valid, axis = 1),
            np.expand_dims(diverse_valid, axis = 1),
            np.expand_dims(y_valid, axis = 1)
        ),
        axis = 1
    ),
    columns = pd.Index(
        list(f"p_{i + 1:d}" for i in range(int(X_train.shape[1]))) +
            [ 'y', 't', 'p', 'd', 's' ],
        dtype = np.str_,
        name = 'feature'
    )
)

mcnnt_col = pd.Index(
    list(f"p_{i + 1:d}" for i in range(int(X_train.shape[1]))),
    dtype = np.str_,
    name = 'feature'
)
pos_col = 'p'
time_col = 't'
diverse_col = 'd'
year_col = 'y'
score_col = 's'

X_col = pd.Index(
    list(mcnnt_col) + [ 'p', 't', 'd', 'y' ],
    dtype = np.str_,
    name = 'feature'
)
X_scalar_col = pd.Index(
    [ 'p', 't', 'd', 'y' ],
    dtype = np.str_,
    name = 'feature'
)
y_col = score_col


### Synthetic Minority Oversampling Technique (SMOTE)


SMOTE is done by oversampling minority bins in histogram. Histogram is first done in $ 2 $ equally wide bins (halves), then $ 3 $ (thirds) and so on; in each iteration, all bins that have less than the maximal number of samples are oversampled using SMOTE. SMOTE skips sampling from a combination of samples $ x $ and $ y $ if their years are more than $ 2 $ apart.


In [ ]:
def SMOTE (feat_df, low = 0, high = 5.5, max_bins = 12):
    for b in range(2, max_bins + 1):
        bin_edges = np.linspace(low, high, num = b + 1, dtype = np.float32)
        bin_edges[-1] += 1.0e-8

        hist, _ = np.histogram(
            feat_df['s'],
            bins = bin_edges,
            density = False
        )

        n_max = np.amax(hist)

        for k in range(b):
            if hist[k] < 2 or hist[k] == n_max:
                continue

            ind = feat_df.index[
                (feat_df['s'] >= bin_edges[k]) &
                (feat_df['s'] < bin_edges[k + 1])
            ]
            years = np.asarray(feat_df.loc[ind, 'y'].values)
            year_diffs = np.absolute(
                np.expand_dims(years, axis = 0) -
                    np.expand_dims(years, axis = 1)
            )
            year_diffs = (year_diffs > 2)

            n_diff = n_max - hist[k]

            I = np.zeros((n_diff, 2), dtype = np.int32)
            while True:
                J = ((I[:, 0] == I[:, 1]) | year_diffs[I[:, 0], I[:, 1]])
                n_g = np.sum(J)

                if not n_g:
                    break

                I[J] = R.choice(hist[k], size = (n_g, 2), replace = True)

            t = R.uniform(0, 1, size = (n_diff, 1))

            gen_df = \
                t * feat_df.loc[ind[I[:, 0]]].reset_index(drop = True) + \
                    (1 - t) * feat_df.loc[ind[I[:, 1]]].reset_index(drop = True)

            feat_df = pd.concat(
                (feat_df, gen_df),
                axis = 0,
                ignore_index = True
            )

    return feat_df


In [ ]:
display(feat_df_train.shape)
display(feat_df_valid.shape)


In [ ]:
feat_df_train = SMOTE(feat_df_train, max_bins = 12)
feat_df_valid = SMOTE(feat_df_valid, max_bins = 12)


In [ ]:
display(feat_df_train.shape)
display(feat_df_valid.shape)


In [ ]:
feat_df_train['y'] = np.around(feat_df_train['y']).astype(np.int32)
feat_df_valid['y'] = np.around(feat_df_valid['y']).astype(np.int32)


### Correlation Reduction


In [ ]:
def observe_correlations (X, **fig_kwargs):
    fig = plt.figure(**fig_kwargs)
    ax = fig.add_subplot(111)

    corr, corr_p = stats.spearmanr(X, axis = 0)
    corr[np.eye(corr.shape[0], dtype = np.bool_)] = 0

    corr_mask = np.triu(np.ones_like(corr, dtype = np.bool_))

    sns.heatmap(
        corr,
        mask = corr_mask,
        vmin = -1,
        vmax = 1,
        center = 0,
        square = True,
        linewidths = 0,
        cmap = 'coolwarm'
    )

    corr_aux = corr.copy(order = 'C')

    corr_aux[~corr_mask] = -np.inf
    i_max = np.unravel_index(np.argmax(corr_aux), shape = corr.shape, order = 'C')

    corr_aux[~corr_mask] = np.inf
    i_min = np.unravel_index(np.argmin(corr_aux), shape = corr.shape, order = 'C')

    print(f"{i_max[0]}, {i_max[1]}: {corr[i_max]:.2%}")
    print(f"{i_min[0]}, {i_min[1]}: {corr[i_min]:.2%}")

    return (corr, fig, ax)


In [ ]:
observe_correlations(
    feat_df_train.loc[:, mcnnt_col],
    figsize = (16, 16)
)

plt.show()


In [ ]:
svd = skldec.TruncatedSVD(
    n_components = X_train.shape[1] - 1,
    random_state = R.integers(np.iinfo(np.int32).max)
)
svd.fit(feat_df_train.loc[:, mcnnt_col])

n = np.argmax(np.cumsum(svd.explained_variance_ratio_) >= 0.95)

M = svd.components_[:n].copy(order = 'C')


In [ ]:
M.shape


In [ ]:
observe_correlations(
    np.matmul(feat_df_train.loc[:, mcnnt_col], M.T),
    figsize = (16, 16)
)

plt.show()


## Model Construction


In [ ]:
def r2_score (y_true, y_pred, sample_weight = None):
    if sample_weight is None:
        sample_weight = K.ones_like(y_true, dtype = 'float32')

    sample_weight = sample_weight / K.sum(sample_weight)

    y_mean = K.sum(sample_weight * y_true)

    SS_res = K.sum(sample_weight * K.square(y_true - y_pred))
    SS_tot = K.maximum(
        K.sum(sample_weight * K.square(y_true - y_mean)),
        K.epsilon()
    )

    return 1 - SS_res / SS_tot

def neg_r2_score (y_true, y_pred, sample_weight = None):
    return -r2_score(y_true, y_pred, sample_weight)


### Hyperparameter Optimisation


In [ ]:
def construct_model (args):
    """Returns inputs, outputs and number of parameters."""
    m_in = 0
    m_out = 0
    n = 0

    inputs = layers.Input(
        shape = M.shape[0] + X_scalar_col.size,
        name = 'input'
    )
    outputs = inputs

    m_in = M.shape[0] + X_scalar_col.size

    j = 1
    for i in range(6):
        m_out = int(round(args[f"hidden_{j:d}"]))

        if m_out > 1:
            outputs = layers.Dense(
                units = m_out,
                activation = 'relu',
                name = f"hidden_{j}"
            )(outputs)

            n += m_out * (m_in + 1)

            m_in = m_out

            j += 1

    outputs = layers.Dense(
        units = 1,
        activation = 'linear',
        name = 'output'
    )(outputs)

    n += m_in + 1

    return (inputs, outputs, n)

def objective (args):
    inputs, outputs, n = construct_model(args)

    if n > 2400:
        return { 'status': hopt.STATUS_FAIL }

    model = models.Model(
        inputs = inputs,
        outputs = outputs,
        name = 'model'
    )

    model.compile(
        optimizer = optimizers.Adadelta(
            learning_rate = 1.0e-2,
            name = 'optimiser'
        ),
        loss = 'mean_squared_error',
        metrics = [ r2_score ]
    )

    history = model.fit(
        x = np.concatenate(
            (
                np.matmul(feat_df_train[mcnnt_col], M.T),
                feat_df_train[X_scalar_col]
            ),
            axis = 1
        ),
        y = feat_df_train[y_col],
        validation_data = (
            np.concatenate(
                (
                    np.matmul(feat_df_valid[mcnnt_col], M.T),
                    feat_df_valid[X_scalar_col]
                ),
                axis = 1
            ),
            feat_df_valid[y_col]
        ),
        shuffle = True,
        batch_size = 256,
        epochs = 100,
        verbose = 0,
        use_multiprocessing = False,
        callbacks = [
            callbacks.TerminateOnNaN(),
            callbacks.EarlyStopping(
                monitor = 'val_loss',
                patience = 4,
                mode = 'min',
                restore_best_weights = True
            ),
            callbacks.ReduceLROnPlateau(
                monitor = 'val_loss',
                patience = 2,
                mode = 'min',
                factor = 0.5
            )
        ]
    )

    loss = np.asanyarray(history.history['val_loss'])
    r2 = np.asanyarray(history.history['val_r2_score'])

    return {
        'status': hopt.STATUS_OK,
        'nparams': n,
        'loss': -math.log(n) * r2[np.argmin(loss)]
    }


In [ ]:
# Setting the number of units to 1 will actually omit the hidden layer.
space = dict(
    (
        f"hidden_{i + 1:d}",
        hopt.hp.quniform(
            f"hidden_{i + 1:d}",
            1,
            int(round(math.ceil(25.0 / (i + 1)))),
            1
        )
    ) for i in range(6)
)


In [ ]:
trials = hopt.Trials()
best = hopt.fmin(
    objective,
    space = space,
    algo = hopt.tpe.suggest,
    max_evals = 500,
    trials = trials
)
with open('hyperopt_trials.pkl', 'wb') as trials_output:
    pickle.dump(trials, trials_output, fix_imports = True)


In [ ]:
best


### Final Model


In [ ]:
inputs, outputs, n = construct_model(best)

model = models.Model(
    inputs = inputs,
    outputs = outputs,
    name = 'model'
)

model.compile(
    optimizer = optimizers.Adadelta(
        learning_rate = 1.0e-3,
        name = 'optimiser'
    ),
    loss = 'mean_squared_error',
    metrics = [
        metrics.RootMeanSquaredError(),
        r2_score,
        metrics.LogCoshError(),
        metrics.CosineSimilarity()
    ]
)


In [ ]:
model.summary()


In [ ]:
utils.plot_model(model)


In [ ]:
history = model.fit(
    x = np.concatenate(
        (
            np.matmul(feat_df_train[mcnnt_col], M.T),
            feat_df_train[X_scalar_col]
        ),
        axis = 1
    ),
    y = feat_df_train[y_col],
    validation_data = (
        np.concatenate(
            (
                np.matmul(feat_df_valid[mcnnt_col], M.T),
                feat_df_valid[X_scalar_col]
            ),
            axis = 1
        ),
        feat_df_valid[y_col]
    ),
    shuffle = True,
    batch_size = 256,
    epochs = 2048,
    verbose = 0,
    use_multiprocessing = False,
    callbacks = [
        callbacks.TerminateOnNaN(),
        callbacks.EarlyStopping(
            monitor = 'val_loss',
            patience = 64,
            mode = 'min',
            restore_best_weights = True
        ),
        callbacks.ReduceLROnPlateau(
            monitor = 'val_loss',
            patience = 2,
            mode = 'min',
            factor = 0.5
        )
    ]
)


In [ ]:
k = len(history.history['loss'])

fig, ax = plt.subplots(nrows = 1, ncols = 2, sharey = False, figsize = (16, 9))

ax[0].plot(
    list(range(1, k + 1)),
    history.history['root_mean_squared_error'][:k]
)
ax[0].plot(
    list(range(1, k + 1)),
    history.history['val_root_mean_squared_error'][:k]
)

ax[1].plot(
    list(range(1, k + 1)),
    history.history['r2_score'][:k]
)
ax[1].plot(
    list(range(1, k + 1)),
    history.history['val_r2_score'][:k]
)

plt.show()


### Testing


In [ ]:
df_test = df.loc[df.year == year].copy(deep = True)
df_test['nscore_pred'] = np.zeros(df_test.shape[0])

pred_test = list()

for i in df.index[df.year == year]:
    X_test = list()
    year_test = list()
    time_test = list()
    pos_test = list()
    diverse_test = list()
    y_test = None

    c = str(np.asarray(df.loc[i, 'country']).item())
    a = np.asarray(df.loc[i, 'year']).item()
    s = np.asarray(df.loc[i, 'nscore']).item()

    x = np.load(os.path.join(arrays_root, str(year), c, 'taggram.npy'))
    t = np.load(os.path.join(arrays_root, str(year), c, 't.npy'))
    p = np.load(os.path.join(arrays_root, str(year), c, 'p.npy'))
    d = np.load(os.path.join(arrays_root, str(year), c, 'diverse.npy'))

    X_test = x
    year_test = np.full((x.shape[0], 1), a, dtype = x.dtype)
    time_test = np.full((x.shape[0], 1), t, dtype = x.dtype)
    pos_test = np.expand_dims(p, axis = 1)
    diverse_test = np.full((x.shape[0], 1), d, dtype = x.dtype)
    y_test = s

    y_test_pred = model.predict(
        np.concatenate(
            (
                np.matmul(X_test, M.T),
                pos_test - 0.5,
                time_test,
                diverse_test,
                year_test - (year - 5.5)
            ),
            axis = 1
        ),
        batch_size = 64
    ).ravel()

    df_test.loc[i, 'nscore_pred'] = np.quantile(y_test_pred, q = 0.25)

    pred_test.append((y_test, y_test_pred))

df_test.sort_values(
    by = [ 'score', 'country' ],
    ascending = [ False, True ],
    inplace = True
)
df_test.reset_index(drop = True, inplace = True)


In [ ]:
fig = plt.figure(figsize = (16, 9))

ax = fig.add_subplot(111)

ax.scatter(
    list(p[0] for p in pred_test),
    list(np.quantile(p[1], q = 0.75) for p in pred_test),
    color = 'C0',
    alpha = 0.1,
    s = 16
)
ax.scatter(
    list(p[0] for p in pred_test),
    list(np.quantile(p[1], q = 0.50) for p in pred_test),
    color = 'C2',
    alpha = 0.1,
    s = 16
)
ax.scatter(
    list(p[0] for p in pred_test),
    list(np.quantile(p[1], q = 0.25) for p in pred_test),
    color = 'C1',
    alpha = 1,
    s = 32
)

plt.show()


In [ ]:
sklmet.r2_score(
    list(p[0] for p in pred_test),
    list(np.quantile(p[1], q = 0.75) for p in pred_test)
)


In [ ]:
df_test['rank_pred'] = myutils.rank_list(df_test['nscore_pred'], mode = 'center')


In [ ]:
df_test
